In [1]:
# pipeline_final_display_fixed.py
# Prompts for a .json or .csv path, runs the full pipeline, and prints ONLY final unique rows in a box.

import re
from pathlib import Path
from datetime import datetime
from collections import Counter
import pandas as pd

# Try to use tabulate for nice boxes; fallback to pandas formatting if unavailable
try:
    from tabulate import tabulate
    def pretty_print(df: pd.DataFrame, limit: int = 50):
        print(tabulate(df.head(limit), headers="keys", tablefmt="fancy_grid", showindex=False))
except Exception:
    def pretty_print(df: pd.DataFrame, limit: int = 50):
        box = df.head(limit).to_string(index=False)
        line = "═" * max(80, len(box.splitlines()[0]) if box else 80)
        print(line)
        print(box)
        print(line)

# ------------------- UTILITIES -------------------
def safe_to_csv(df, path: Path, **kwargs) -> Path:
    """Save to CSV with fallbacks to avoid PermissionError."""
    candidates = [path] + [path.with_name(f"{path.stem}_unlocked{i}{path.suffix}") for i in range(1, 4)]
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    candidates.append(path.with_name(f"{path.stem}_{ts}{path.suffix}"))
    for p in candidates:
        try:
            df.to_csv(p, index=False, **kwargs)
            return p
        except PermissionError:
            continue
    raise PermissionError("Unable to save file (file may be open).")

def coerce_to_ts(series):
    s = pd.to_datetime(series, errors="coerce", utc=True)
    if s.notna().any():
        return s
    num = pd.to_numeric(series, errors="coerce")
    s = pd.to_datetime(num, unit="ms", errors="coerce", utc=True)
    if s.notna().any():
        return s
    return pd.to_datetime(num, unit="s", errors="coerce", utc=True)

def norm(v):
    if pd.isna(v): return pd.NA
    s = str(v).strip()
    return pd.NA if s == "" else s

def first_nonempty(row, keys):
    for k in keys:
        if k in row:
            val = norm(row[k])
            if not pd.isna(val):
                return val
    return pd.NA

def safe_eq(a, b):
    if pd.isna(a) and pd.isna(b): return True
    if pd.isna(a) or pd.isna(b): return False
    return a == b

# ------------------- STEP 1: JSON TO CSV -------------------
TESTCASE_DELIMS = ["]}}]}", "]}}],", "]}}]},", "]}},"]

def parse_jsonish_to_csv(path: Path) -> Path:
    out = path.with_suffix(".csv")
    text = Path(path).read_text(encoding="utf-8", errors="ignore")
    delim = next((d for d in TESTCASE_DELIMS if d in text), TESTCASE_DELIMS[0])
    chunks = text.split(delim)

    re_evar = re.compile(r'"(eVar\d+)"\s*:\s*"([^"]*)"')
    re_prop = re.compile(r'"(prop\d+)"\s*:\s*"([^"]*)"')
    re_events = re.compile(r'"events"\s*:\s*"([^"]*)"')
    re_track = re.compile(r'"trackAction"\s*:\s*"([^"]*)"|"trackaction"\s*:\s*"([^"]*)"')  # 2 groups
    re_prod = re.compile(r'"product-string"\s*:\s*"([^"]*)"')
    re_msg  = re.compile(r'"messageReceivedTime"\s*:\s*\{\s*"date"\s*:\s*"([^"]*)"')
    re_impr = re.compile(r'"impressionData"\s*:\s*"([^"]*)"')
    re_cust = re.compile(r'"custom_event_name"\s*:\s*"([^"]*)"')

    rows = []
    for i, c in enumerate(chunks, 1):
        if not c.strip(): 
            continue
        row = {"test_case_id": i}

        for k, v in re_evar.findall(c):
            row[k] = v
        for k, v in re_prop.findall(c):
            row[k] = v

        m = re_events.search(c)
        if m: row["events"] = m.group(1)

        m = re_prod.search(c)
        if m: row["product-string"] = m.group(1)

        m = re_track.search(c)
        if m: row["trackaction"] = (m.group(1) or m.group(2))  # safe for 2-group pattern only

        m = re_cust.search(c)
        if m: row["custom_event_name"] = m.group(1)

        m = re_impr.search(c)
        if m: row["impressionData"] = m.group(1)

        m = re_msg.search(c)
        if m: row["messageReceivedTime"] = m.group(1)

        rows.append(row)

    df = pd.DataFrame(rows)
    return safe_to_csv(df, out, encoding="utf-8")

# ------------------- STEP 2: MODULE IDENTIFY -------------------
MODULE_KEYWORDS = {
    "Home page":{"custom":["home page","onboarding","notification response","content link click"],"action":["home page","onboarding","notification response"]},
    "Search":{"custom":["search"],"action":["search"]},
    "Navigation":{"custom":["nav","navigation"],"action":["nav","navigation"]},
    "My account":{"custom":["sign-in","my account","login","account sign-in","store"],"action":["sign-in","my account","login","account sign-in","store"]},
    "Wishlist":{"custom":["wishlist"],"action":["wishlist"]},
    "PDP":{"custom":["pdp","product","product click"],"action":["pdp","product","product click"]},
    "Checkout":{"custom":["checkout","step up login","login & checkout","shipping","contact info","order confirmation","payment"],
                "action":["checkout","step up login","login & checkout","payment"]},
    "App Specific":{"custom":["app specific","app-specific"],"action":["app specific","app-specific"]},
}

def _norm_text(s): return "" if pd.isna(s) else str(s).strip().lower()
def modules_in_text(t):
    t = _norm_text(t)
    return {m for m,d in MODULE_KEYWORDS.items() if any(w in t for w in d["custom"]+d["action"])}

def classify_modules(path: Path) -> Path:
    df = pd.read_csv(path, dtype=str).replace({"": pd.NA})

    mods = []
    for _, r in df.iterrows():
        ce, ta = r.get("custom_event_name", ""), r.get("trackaction", "")
        # Special rule: tab bar bottom nav => Navigation only
        if "tab bar bottom nav" in _norm_text(ta):
            mods.append("Navigation")
            continue
        hits = modules_in_text(ce) | modules_in_text(ta)
        mods.append(", ".join(sorted(hits)) if hits else "Unknown")
    df["module"] = mods

    # eVar216 zone override for Unknown: SB_ZONE -> Bag, HP_ZONE -> Home page
    if "eVar216" in df.columns:
        for i, r in df.iterrows():
            if df.at[i, "module"] == "Unknown":
                e216 = _norm_text(r.get("eVar216", ""))
                if "sb_zone" in e216:
                    df.at[i, "module"] = "Bag"
                elif "hp_zone" in e216:
                    df.at[i, "module"] = "Home page"

    out = path.with_name(f"{path.stem}_modules.csv")
    return safe_to_csv(df, out, encoding="utf-8")

# ------------------- STEP 3: DUPLICATES (adjacent; compare families) -------------------
def rows_match(prev, cur, evars, props):
    for c in evars + props:
        if not safe_eq(norm(prev.get(c, pd.NA)), norm(cur.get(c, pd.NA))):
            return False
    if not safe_eq(first_nonempty(prev, ["custom_event_name"]), first_nonempty(cur, ["custom_event_name"])): return False
    if not safe_eq(first_nonempty(prev, ["trackaction", "trackAction"]), first_nonempty(cur, ["trackaction", "trackAction"])): return False
    if not safe_eq(first_nonempty(prev, ["product-string", "product", "product_string", "productString"]),
                   first_nonempty(cur,  ["product-string", "product", "product_string", "productString"])): return False
    if not safe_eq(first_nonempty(prev, ["events"]), first_nonempty(cur, ["events"])): return False
    if not safe_eq(first_nonempty(prev, ["impressionData", "impression_data"]),
                   first_nonempty(cur,  ["impressionData", "impression_data"])): return False
    return True

def detect_duplicates(path: Path) -> Path:
    df = pd.read_csv(path, dtype=str).replace({"": pd.NA})
    # Use messageReceivedTime when present
    ts = df.get("messageReceivedTime", pd.NaT)
    df["__ts"] = coerce_to_ts(ts)
    df = df.sort_values(by=["__ts"], kind="mergesort").reset_index(drop=True)

    evars = [c for c in df.columns if c.lower().startswith("evar")]
    props = [c for c in df.columns if c.lower().startswith("prop")]

    df["dup_status"] = "unique"
    for i in range(1, len(df)):
        if rows_match(df.loc[i-1], df.loc[i], evars, props):
            df.at[i, "dup_status"] = "duplicate"

    out = path.with_name(f"{path.stem}_duplicate_calls.csv")
    return safe_to_csv(df[df["dup_status"] == "duplicate"], out, encoding="utf-8")

# ------------------- STEP 4: UNIQUE ROWS (from duplicates) -------------------
IGNORE = {"__ts", "messageReceivedTime", "test_case_id", "module_chain", "_nav_for_next_str", "initial_test_case_id"}

def unique_from_duplicates(path: Path) -> Path:
    df = pd.read_csv(path, dtype=str).replace({"": pd.NA})
    # If no duplicates found file is empty: handle gracefully
    if df.empty:
        out = path.with_name(f"{path.stem}_unique_rows.csv")
        safe_to_csv(df, out, encoding="utf-8")
        print("\n📊 FINAL UNIQUE ROWS (no duplicates found)\n")
        pretty_print(df)
        print(f"\n✅ Unique rows saved to: {out} | Total: {len(df)} rows")
        return out

    # Standardize whitespace
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].astype("string").str.strip()

    cols = [c for c in df.columns if c not in IGNORE]
    uniq = df.drop_duplicates(subset=cols, keep="first")
    out = path.with_name(f"{path.stem}_unique_rows.csv")
    safe_to_csv(uniq, out, encoding="utf-8")

    print("\n" + "═"*100)
    print("📊 FINAL UNIQUE ROWS".center(100))
    print("═"*100 + "\n")
    pretty_print(uniq)
    print(f"\n✅ Unique rows saved to: {out} | Total: {len(uniq)} rows")
    print("═"*100)
    return out

# ------------------- MAIN -------------------
if __name__ == "__main__":
    user_path = input("📂 Enter file path (.json or .csv): ").strip().strip('"')
    if not user_path:
        raise SystemExit("❌ No file provided.")
    p = Path(user_path)
    if not p.exists():
        raise SystemExit(f"❌ File not found: {p}")

    # Start from JSON or CSV
    if p.suffix.lower() == ".json":
        csv_base = parse_jsonish_to_csv(p)
    else:
        csv_base = p

    csv_mods = classify_modules(csv_base)
    csv_dups = detect_duplicates(csv_mods)
    unique_from_duplicates(csv_dups)


📂 Enter file path (.json or .csv): D:\Office Data\Downloads\Bcom_testtt.json

════════════════════════════════════════════════════════════════════════════════════════════════════
                                        📊 FINAL UNIQUE ROWS                                         
════════════════════════════════════════════════════════════════════════════════════════════════════

╒════════════════╤═════════╤═════════╤══════════╤════════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤═══════════╤════════════════════════════════════════════════════════╤══════════════════════════════════════╤═══════════╤═════════╤═════════╤════════════════════════════════════════╤═════════╤══════════╤════════════════╤════════════════════════════════════════════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════